In [1]:
import pandas as pd
import numpy as np
import time

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from scipy import stats
from sklearn.linear_model import LinearRegression
from tqdm.notebook import tqdm

pd.set_option("display.width", 500)
pd.set_option("display.max_columns", 100)

In [2]:
DAY_INTERVAL = 7
MAX_DAYS = 735
DAY_LIMITS = list(range(DAY_INTERVAL, MAX_DAYS + 1, DAY_INTERVAL))

In [3]:
def load_users(users_path: str, super_users_path: str) -> pd.DataFrame:
    """
    Load and preprocess user data, excluding super users from regular users.
    Flags optional fields as True/False based on their presence.
    """
    start_time = time.time()
    # Columns to load
    user_cols = ["Id", "CreationDate", "AboutMe", "Location", "WebsiteUrl"]

    # Load user data
    users = pd.read_parquet(users_path, columns=user_cols)
    super_users = pd.read_parquet(super_users_path, columns=user_cols)
    print(f"{time.time() - start_time:.2f}s: Users and Super Users loaded.")

    # Mark super users
    super_users["SuperUser"] = True
    users["SuperUser"] = False

    # Remove super users from regular users using set for faster lookup
    super_user_ids = set(super_users["Id"])
    users = users[~users["Id"].isin(super_user_ids)]

    # Concatenate regular and super users
    all_users = pd.concat([users, super_users], ignore_index=True)
    print(f"{time.time() - start_time:.2f}s: Super Users removed from regular users and concatenated.")

    # Convert CreationDate to datetime
    all_users["CreationDate"] = pd.to_datetime(all_users["CreationDate"], errors="coerce")

    # Filter out users with non-positive Id
    all_users = all_users[all_users["Id"] > 0]

    # Implement flags for optional fields
    optional_fields = ["AboutMe", "Location", "WebsiteUrl"]
    for field in optional_fields:
        all_users[f"{field}_Present"] = all_users[field].notnull()

    # Drop the original optional fields if not needed
    all_users.drop(columns=optional_fields, inplace=True)

    print(f"{time.time() - start_time:.2f}s: Optional fields flagged.")

    return all_users


def load_activity(super_activity_path: str, non_super_activity_path: str) -> pd.DataFrame:
    """
    Load and preprocess activity data, renaming and concatenating super and non-super user activities.
    """
    start_time = time.time()
    # Columns to load
    activity_cols = ["UserId", "CreationDate", "ActionType"]

    # Load activity data
    super_activity = pd.read_parquet(super_activity_path, columns=activity_cols)
    non_super_activity = pd.read_parquet(non_super_activity_path, columns=activity_cols)
    print(f"{time.time() - start_time:.2f}s: Super and Non-Super Activity loaded.")

    # Rename 'CreationDate' to 'ActivityDate' and convert to datetime
    super_activity.rename(columns={"CreationDate": "ActivityDate"}, inplace=True)
    non_super_activity.rename(columns={"CreationDate": "ActivityDate"}, inplace=True)

    super_activity["ActivityDate"] = pd.to_datetime(super_activity["ActivityDate"], errors="coerce")
    non_super_activity["ActivityDate"] = pd.to_datetime(non_super_activity["ActivityDate"], errors="coerce")

    # Concatenate activities
    activity = pd.concat([super_activity, non_super_activity], ignore_index=True)
    print(f"{time.time() - start_time:.2f}s: Activities concatenated.")

    # Filter out activities with non-positive UserId
    activity = activity[activity["UserId"] > 0]
    print(f"{time.time() - start_time:.2f}s: Activities filtered for positive UserId.")

    return activity


def merge_user_activity(activity: pd.DataFrame, users: pd.DataFrame) -> pd.DataFrame:
    """
    Merge activity data with user data on UserId.
    """
    start_time = time.time()
    user_activity = activity.merge(
        users, left_on="UserId", right_on="Id", how="left"
    ).drop(columns=["Id"])
    print(f"{time.time() - start_time:.2f}s: User and Activity data merged.")
    return user_activity


def process_user_activity(user_activity: pd.DataFrame, days_limit: int) -> pd.DataFrame:
    """
    Calculate DaysSinceRegistration and filter activities within the specified days limit.
    """
    start_time = time.time()
    # Calculate DaysSinceRegistration
    user_activity["DaysSinceRegistration"] = (
        user_activity["ActivityDate"] - user_activity["CreationDate"]
    ).dt.days

    # Filter activities within the limit
    user_activity_limited = user_activity[
        user_activity["DaysSinceRegistration"] <= days_limit
    ].dropna(subset=["UserId", "SuperUser"])
    print(
        f"{time.time() - start_time:.2f}s: User activity processed for days_limit={days_limit}."
    )

    return user_activity_limited


def compute_action_counts(user_activity_limited: pd.DataFrame) -> pd.DataFrame:
    """
    Compute the count of each ActionType for each UserId and SuperUser status.
    """
    start_time = time.time()
    action_counts = (
        user_activity_limited.groupby(["UserId", "SuperUser", "ActionType"])
        .size()
        .reset_index(name="ActionCount")
    )
    print(f"{time.time() - start_time:.2f}s: Action counts computed.")
    return action_counts


def create_full_cross_product(
    action_counts: pd.DataFrame, user_activity_limited: pd.DataFrame, days_limit: int
) -> pd.DataFrame:
    """
    Create a full cross-product of UserId, SuperUser, and ActionType to ensure all combinations exist.
    Merge with action counts and fill missing values with 0.
    Pivot the data so that each ActionType becomes a separate column.
    """
    start_time = time.time()
    # Unique users and actions, including optional field flags
    unique_users = user_activity_limited[
        ["UserId", "SuperUser", "AboutMe_Present", "Location_Present", "WebsiteUrl_Present"]
    ].drop_duplicates()
    unique_actions = user_activity_limited["ActionType"].unique()

    # Create cross product using a key
    unique_users["key"] = 1
    unique_actions_df = pd.DataFrame(unique_actions, columns=["ActionType"])
    unique_actions_df["key"] = 1

    full_index = pd.merge(unique_users, unique_actions_df, on="key").drop("key", axis=1)

    # Merge with action counts
    result = pd.merge(
        full_index, action_counts, on=["UserId", "SuperUser", "ActionType"], how="left"
    )

    # Fill missing ActionCount with 0
    result["ActionCount"] = result["ActionCount"].fillna(0).astype(int)

    # Assign DaysSinceRegistration
    result["DaysSinceRegistration"] = days_limit

    # Pivot the result so that each ActionType becomes a separate column
    pivoted_result = result.pivot_table(
        index=["UserId", "SuperUser", "DaysSinceRegistration",
               "AboutMe_Present", "Location_Present", "WebsiteUrl_Present"],
        columns="ActionType",
        values="ActionCount",
        fill_value=0,
    ).reset_index()

    # Flatten the MultiIndex columns after pivot
    pivoted_result.columns = ["UserId", "SuperUser", "DaysSinceRegistration",
                              "AboutMe_Present", "Location_Present", "WebsiteUrl_Present"] + [
        f"{col}_Count" for col in pivoted_result.columns if col not in ["UserId", "SuperUser", "DaysSinceRegistration",
                                                                          "AboutMe_Present", "Location_Present",
                                                                          "WebsiteUrl_Present"]
    ]

    print(
        f"{time.time() - start_time:.2f}s: Full cross-product and pivot created for days_limit={days_limit}."
    )

    return pivoted_result

In [ ]:
total_start_time = time.time()

# File paths
users_path = "../data/Users.parquet"
super_users_path = "../data/Users_Super.parquet"
super_activity_path = "../data_users/actions_cleaned_SuperUserIds.parquet"
non_super_activity_path = (
    "../data_users/actions_cleaned_NonSuperUserIdsSample_0.1.parquet"
)

# Load and preprocess user data
users_start_time = time.time()
users = load_users(users_path, super_users_path)
print("\nUsers DataFrame:")
print(users.shape)
print(users.head())
print(f"{time.time() - users_start_time:.2f}s: Users loaded and preprocessed.")

# Load and preprocess activity data
activity_start_time = time.time()
activity = load_activity(super_activity_path, non_super_activity_path)
print("\nActivity DataFrame:")
print(activity.shape)
print(activity.head())
print(f"{time.time() - activity_start_time:.2f}s: Activity loaded and preprocessed.")

# Merge user and activity data
merge_start_time = time.time()
user_activity = merge_user_activity(activity, users)
print("\nUser-Activity DataFrame:")
print(user_activity.shape)
print(user_activity.head())
print(f"{time.time() - merge_start_time:.2f}s: User-Activity merged.")

# Initialize a list to collect results for all day limits
all_results = []

# Iterate over each day limit with a progress bar
for days_limit in tqdm(DAY_LIMITS, desc="Processing Day Limits"):
    print(f"\nProcessing for DaysSinceRegistration = {days_limit}")
    step_start_time = time.time()

    # Process user activity within the specified days limit
    user_activity_limited = process_user_activity(user_activity, days_limit)
    print("Limited User-Activity DataFrame:")
    print(user_activity_limited.shape)
    print(user_activity_limited.head())
    print(f"{time.time() - step_start_time:.2f}s: User-Activity limited.")

    # Compute action counts
    action_counts = compute_action_counts(user_activity_limited)
    print("\nAction Counts DataFrame:")
    print(action_counts.shape)
    print(action_counts.head())
    print(f"{time.time() - step_start_time:.2f}s: Action counts computed.")

    # Create full cross-product and pivoted result for current day limit
    result = create_full_cross_product(action_counts, user_activity_limited, days_limit)
    print("\nFinal Pivoted Result DataFrame for current day limit:")
    print(result.shape)
    print(result.head())
    print(
        f"{time.time() - step_start_time:.2f}s: Full cross-product and pivot created for current day limit."
    )

    # Append the pivoted result to the list
    all_results.append(result)
    print(
        f"{time.time() - step_start_time:.2f}s: Processing for days_limit={days_limit} completed."
    )

# Concatenate all results
concatenation_start_time = time.time()
final_result = pd.concat(all_results, ignore_index=True)
print(f"\nAll results concatenated. Final Result DataFrame shape: {final_result.shape}")
print(final_result.head())
print(f"{time.time() - concatenation_start_time:.2f}s: All results concatenated.")

# Save the final result to Parquet
save_start_time = time.time()
output_parquet_path = f"../data_users/results_multiple_days_{MAX_DAYS}.parquet"
final_result.to_parquet(output_parquet_path, index=False)
print(
    f"{time.time() - save_start_time:.2f}s: Final results saved to {output_parquet_path}."
)

print(f"\nTotal processing time: {time.time() - total_start_time:.2f}s.")

17.70s: Users and Super Users loaded.
19.78s: Super Users removed from regular users and concatenated.
34.26s: Optional fields flagged.

Users DataFrame:
(22484218, 6)
    Id            CreationDate  SuperUser  AboutMe_Present  Location_Present  WebsiteUrl_Present
17   2 2008-07-31 14:22:31.287      False             True              True                True
18   3 2008-07-31 14:22:31.287      False             True              True                True
19   8 2008-07-31 21:33:24.057      False             True             False               False
20  10 2008-07-31 21:57:06.240      False             True              True                True
21  11 2008-08-01 00:59:11.147      False            False             False               False
34.89s: Users loaded and preprocessed.
24.01s: Super and Non-Super Activity loaded.
56.13s: Activities concatenated.
56.59s: Activities filtered for positive UserId.

Activity DataFrame:
(65242970, 3)
   UserId        ActivityDate ActionType
0     78

Processing Day Limits:   0%|          | 0/105 [00:00<?, ?it/s]


Processing for DaysSinceRegistration = 7
1.69s: User activity processed for days_limit=7.
Limited User-Activity DataFrame:
(2342338, 9)
    UserId        ActivityDate ActionType            CreationDate  SuperUser  AboutMe_Present  Location_Present  WebsiteUrl_Present  DaysSinceRegistration
1     3434 2008-09-02 03:44:26     Answer 2008-08-28 15:06:22.043       True             True              True                True                      4
5     2859 2008-09-02 04:12:00     Answer 2008-08-25 15:50:17.907       True             True              True                True                      7
8     3742 2008-09-02 04:18:41     Answer 2008-08-30 14:08:11.733       True             True              True                True                      2
9     3631 2008-09-02 04:23:34     Answer 2008-08-29 16:06:08.497       True             True              True               False                      3
10    3153 2008-09-02 04:36:54   Question 2008-08-27 02:45:05.220       True            